# Conflict-Free Scheduling System Tutorial

This tutorial demonstrates how to use the Conflict-Free Scheduling System to create optimal academic schedules. We'll walk through the steps of importing modules, setting up activities, running the scheduling algorithms, and visualizing the results.

## Overview of Components

The system includes four core scheduling algorithms:
1. Graph Coloring - Models conflicts as graph edges, assigns time slots as colors
2. Dynamic Programming - Optimal weighted activity selection with memoization
3. Backtracking - Exhaustive search with pruning for optimal solutions
4. Genetic Algorithm - Population-based evolutionary optimization

## 1. Import Project Modules

Let's start by importing the necessary modules and classes from our project.

In [ ]:
import sys
import os
import time
from datetime import datetime

# Add the project root to the Python path
sys.path.append(os.getcwd())

# Import core components
try:
    from src import ConflictFreeScheduler, Activity, Task
    from src.utils import FileParser, PDFGenerator, AcademicPDFGenerator
    from src.algorithms import GraphColoringScheduler, DynamicProgrammingScheduler
    from src.algorithms import BacktrackingScheduler, GeneticAlgorithmScheduler
    print("✅ Successfully imported project modules")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running this notebook from the project root directory")

## 2. Load Sample Data

Now we'll create or load some sample activities for scheduling. These activities represent classes with their time slots and weights (priority/credits).

In [ ]:
# Function to load or create sample data
def load_sample_data():
    """Load or create sample academic activities"""
    
    # Check if we have sample data in CSV format
    sample_file = "data/demo_activities.csv"
    if os.path.exists(sample_file):
        print(f"📂 Loading activities from {sample_file}")
        return FileParser.parse_csv(sample_file)
    
    # Otherwise, create sample activities
    print("📝 Creating sample academic activities")
    activities = [
        Activity(1, 0, 90, 3.0, "Programming Fundamentals", "Room 101"),
        Activity(2, 100, 190, 3.0, "Data Structures", "Room 102"),
        Activity(3, 50, 140, 1.0, "Programming Lab", "Lab 1"),  # Conflicts with Activity 1
        Activity(4, 200, 290, 3.0, "Algorithms", "Room 103"),
        Activity(5, 150, 240, 3.0, "Digital Logic Design", "Room 104"),
        Activity(6, 300, 390, 3.0, "Database Systems", "Room 101"),
        Activity(7, 270, 360, 3.0, "Operating Systems", "Room 102"),  # Conflicts with Activity 6
        Activity(8, 400, 490, 1.5, "Database Lab", "Lab 2"),
        Activity(9, 500, 590, 3.0, "Computer Networks", "Room 103"),
        Activity(10, 550, 640, 3.0, "Software Engineering", "Room 104"),  # Conflicts with Activity 9
    ]
    return activities

# Load the activities
activities = load_sample_data()

# Display information about our sample activities
print(f"\n📊 Loaded {len(activities)} activities")
print("\nSample activities:")
for i, activity in enumerate(activities[:5]):  # Show first 5
    print(f"  {i+1}. {activity}")
print("  ...")

# Save to CSV for future use (optional)
if not os.path.exists("data/demo_activities.csv"):
    os.makedirs("data", exist_ok=True)
    FileParser.write_csv(activities, "data/demo_activities.csv")
    print("\n✅ Saved sample activities to data/demo_activities.csv")

## 3. Initialize ConflictFreeScheduler

Now we'll create an instance of the ConflictFreeScheduler class to manage our scheduling operations.

In [ ]:
# Initialize the scheduler
scheduler = ConflictFreeScheduler()
print("✅ Initialized ConflictFreeScheduler")

# Check for conflicts in our sample activities
conflicts = scheduler.find_conflicts(activities)
print(f"\n⚠️ Found {len(conflicts)} conflicts in the sample data:")

for (a1, a2) in conflicts[:3]:  # Show first 3 conflicts
    print(f"  - {a1.name} (ID: {a1.id}, Time: {a1.start}-{a1.end}) conflicts with")
    print(f"    {a2.name} (ID: {a2.id}, Time: {a2.start}-{a2.end})")
    
if len(conflicts) > 3:
    print(f"  ... and {len(conflicts) - 3} more conflicts")

## 4. Generate Conflict-Free Schedule

Now let's use each of our algorithms to generate conflict-free schedules and compare their performance.

In [ ]:
# Function to run all algorithms and compare results
def run_all_algorithms(activities):
    """Run all scheduling algorithms and compare results"""
    results = {}
    execution_times = {}
    
    algorithms = {
        "Graph Coloring": scheduler.graph_coloring_schedule,
        "Dynamic Programming": scheduler.dp_schedule,
        "Backtracking": scheduler.backtracking_schedule,
        "Genetic Algorithm": scheduler.genetic_algorithm_schedule
    }
    
    for name, algorithm in algorithms.items():
        print(f"\n🔄 Running {name} algorithm...")
        start_time = time.time()
        results[name] = algorithm(activities)
        execution_time = (time.time() - start_time) * 1000  # ms
        execution_times[name] = execution_time
        
        print(f"  ✅ Completed in {execution_time:.2f}ms")
        print(f"  📊 Scheduled {len(results[name])}/{len(activities)} activities " + 
              f"with total weight {scheduler.calculate_total_weight(results[name]):.2f}")
    
    return results, execution_times

# Run all algorithms and get results
results, execution_times = run_all_algorithms(activities)

## 5. Visualize the Schedule

Let's create some visualizations to compare the results of different algorithms.

In [ ]:
# Function to create a comparison table
def display_comparison_table(results, execution_times):
    """Display a formatted comparison table of algorithm results"""
    print("\n📊 ALGORITHM COMPARISON")
    print("=" * 80)
    print(f"{'Algorithm':<20} {'Activities':<12} {'Total Weight':<15} {'Efficiency %':<12} {'Time (ms)':<12}")
    print("-" * 80)
    
    for name, result in results.items():
        weight = scheduler.calculate_total_weight(result)
        efficiency = len(result) / len(activities) * 100
        time_ms = execution_times[name]
        
        print(f"{name:<20} {len(result):<12} {weight:<15.2f} {efficiency:<12.1f} {time_ms:<12.2f}")

# Display comparison table
display_comparison_table(results, execution_times)

# Function to display the best schedule
def display_best_schedule(results):
    """Display the best schedule based on total weight"""
    best_algo = max(results.keys(), key=lambda k: scheduler.calculate_total_weight(results[k]))
    best_result = results[best_algo]
    
    print(f"\n🏆 Best algorithm: {best_algo}")
    print(f"📋 Schedule details ({len(best_result)}/{len(activities)} activities scheduled):")
    print("-" * 80)
    print(f"{'ID':<5} {'Start':<8} {'End':<8} {'Weight':<8} {'Room':<10} Course Name")
    print("-" * 80)
    
    for activity in sorted(best_result, key=lambda a: a.start):
        print(f"{activity.id:<5} {activity.start:<8} {activity.end:<8} {activity.weight:<8.1f} " +
              f"{activity.room:<10} {activity.name}")

# Display the best schedule
display_best_schedule(results)

## 6. Generate PDF Output

Let's generate PDF output for the best schedule using the built-in PDF generator.

In [ ]:
# Find the best result
best_algo = max(results.keys(), key=lambda k: scheduler.calculate_total_weight(results[k]))
best_result = results[best_algo]

# Create PDF generator
pdf_gen = PDFGenerator()

# Generate basic schedule HTML
html_file = pdf_gen.generate_schedule_html(
    best_result,
    title=f"Schedule - {best_algo} Algorithm",
    filename=f"schedule_{best_algo.lower().replace(' ', '_')}.html"
)
print(f"✅ Generated basic schedule HTML: {html_file}")

# Generate academic schedule using enhanced PDF generator
academic_pdf = AcademicPDFGenerator()
academic_html = academic_pdf.generate_academic_schedule(
    best_result,
    batch_code="BCSE24",
    section="A",
    semester="Summer 2025"
)
print(f"✅ Generated academic schedule HTML: {academic_html}")

print("\nView the generated HTML files in your browser for a professional visualization.")

## 7. Run Unit Tests

Finally, let's run the unit tests to ensure our scheduler is working correctly.

In [ ]:
import unittest
import importlib.util

# Function to run unit tests
def run_tests():
    """Run unit tests for the scheduler"""
    # Check if pytest is available
    pytest_spec = importlib.util.find_spec("pytest")
    if pytest_spec is not None:
        import pytest
        print("🧪 Running tests with pytest...")
        pytest.main(["-v", "tests/test_scheduler.py"])
    else:
        # Fall back to unittest
        print("🧪 Running tests with unittest...")
        # Discover and run tests
        test_loader = unittest.TestLoader()
        test_suite = test_loader.discover("tests", pattern="test_*.py")
        test_runner = unittest.TextTestRunner(verbosity=2)
        test_runner.run(test_suite)

# Run the tests
run_tests()

## Conclusion

In this tutorial, we've demonstrated how to:
1. Import and use the core components of the Conflict-Free Scheduling System
2. Load and manage activities (courses, labs, etc.)
3. Run all four scheduling algorithms and compare their performance
4. Visualize the results and generate professional output
5. Verify system correctness with unit tests

The system is now fully converted to Python and ready for academic scheduling use.

In [ ]:
# Let's create a visualization to compare algorithm performance
import matplotlib.pyplot as plt
import numpy as np

# Check if matplotlib is available
try:
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Data preparation
    algo_names = list(results.keys())
    scheduled_counts = [len(results[algo]) for algo in algo_names]
    weights = [scheduler.calculate_total_weight(results[algo]) for algo in algo_names]
    exec_times = [execution_times[algo] for algo in algo_names]
    efficiency = [count / len(activities) * 100 for count in scheduled_counts]
    
    # Create bar chart for scheduled activities
    ax1.bar(algo_names, scheduled_counts, color='skyblue')
    ax1.set_title('Number of Scheduled Activities')
    ax1.set_ylabel('Activities')
    ax1.set_ylim(0, len(activities) * 1.1)
    
    # Add efficiency percentage labels
    for i, (count, eff) in enumerate(zip(scheduled_counts, efficiency)):
        ax1.annotate(f'{eff:.1f}%', 
                    xy=(i, count), 
                    xytext=(0, 5),
                    textcoords='offset points',
                    ha='center')
    
    # Create bar chart for execution times
    ax2.bar(algo_names, exec_times, color='salmon')
    ax2.set_title('Execution Time (ms)')
    ax2.set_ylabel('Time (milliseconds)')
    
    # Finalize and display the plot
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Performance visualization created successfully")
except ImportError:
    print("\n⚠️ Matplotlib not installed. Install it with: pip install matplotlib")
    print("   For a full setup including visualization, run: ./run.sh setup")
except Exception as e:
    print(f"\n⚠️ Error creating visualization: {e}")
    print("   Make sure you have the latest version of matplotlib installed")